# 📂 Google Drive Shared Folder Cloner
### **Make shared files your own without downloading.**

This notebook provides a tool to duplicate a folder shared with you (from the *"Shared with me"* section) into your own *"My Drive"*.

**Why use this?**
* **Ownership:** The original owner can delete shared files at any time. This tool creates an independent copy where **you are the owner**.
* **Speed:** It runs entirely on Google's servers. You do not need to download gigabytes of data to your PC and re-upload them.
* **Preservation:** It keeps the folder structure and file names exactly as they are.

---

### ⚠️ CRITICAL STEP: Before you begin
Google Colab cannot directly access the *"Shared with me"* tab. You must create a "bridge" first:

1.  Go to your Google Drive and open **"Shared with me"**.
2.  Right-click the folder you want to copy.
3.  Select **Organize** $\rightarrow$ **Add shortcut**.
4.  Choose **"My Drive"** as the location and click **Add**.

*> Once you have done this, the shared folder will appear in your Drive as a shortcut, and this tool will be able to see it.*

---

### 🚀 How to run this notebook
1.  **Step 1:** Run the setup cell to connect your Google Drive.
2.  **Step 2:** Use the file selector to pick the **Shortcut** (Source) and a **Destination** folder.
3.  **Step 3:** Watch the real-time logs as the script copies your files.

### 🛠️ Step 1: Install Tools & Mount Drive
**Instructions:** Run this cell once to install the file selector library and connect to your Google Drive.

In [ ]:
# Install the necessary library silently
!pip install ipyfilechooser -q

from google.colab import drive
from ipyfilechooser import FileChooser
from IPython.display import display
import ipywidgets as widgets
import os
import shutil

# Mount Google Drive
drive.mount('/content/drive')
print("✅ Setup complete and Drive mounted.")

### 📂 Step 2: Select Folders
**Instructions:**
1. Run this cell to see the file pickers.
2. Select the **Source** folder (the one you want to copy).
3. Select the **Target Parent** folder (where the copy will be placed).
4. Type a **New Name** for the folder.
5. Click **"Confirm Settings"**.

In [ ]:
# --- 1. SOURCE SELECTOR ---
print("📂 Please select the SOURCE folder to copy:")
fc_source = FileChooser('/content/drive/MyDrive')
fc_source.show_only_dirs = True  # Show folders only
fc_source.title = '<b>Source Folder (What to copy?)</b>'
display(fc_source)

print("-" * 60)

# --- 2. TARGET PARENT SELECTOR ---
print("📂 Please select the TARGET PARENT folder:")
fc_target_parent = FileChooser('/content/drive/MyDrive')
fc_target_parent.show_only_dirs = True
fc_target_parent.title = '<b>Target Location (Where to put it?)</b>'
display(fc_target_parent)

# --- 3. NEW NAME INPUT ---
print("-" * 60)
text_new_name = widgets.Text(
    value='Backup_Folder',
    placeholder='Type folder name',
    description='<b>New Name:</b>',
    disabled=False
)
display(text_new_name)

# --- 4. ACTION BUTTON ---
btn_process = widgets.Button(
    description='Confirm Settings & Show Paths',
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    icon='check',
    layout=widgets.Layout(width='300px')
)

output = widgets.Output()

def on_button_clicked(b):
    with output:
        output.clear_output()
        if fc_source.selected and fc_target_parent.selected:
            source_path = fc_source.selected
            # Target path = Selected Parent + New Name
            target_path = os.path.join(fc_target_parent.selected, text_new_name.value)

            print(f"✅ SETTINGS CONFIRMED!")
            print(f"--------------------------------------------------")
            print(f"📍 SOURCE: {source_path}")
            print(f"📍 TARGET: {target_path}")
            print(f"--------------------------------------------------")
            print("⚠️ You can now run the next cell to start copying.")

            # Store variables globally so the next cell can see them
            global FINAL_SOURCE_PATH, FINAL_TARGET_PATH
            FINAL_SOURCE_PATH = source_path
            FINAL_TARGET_PATH = target_path
        else:
            print("❌ ERROR: Please select both a Source and a Target folder.")

btn_process.on_click(on_button_clicked)
display(btn_process, output)

### 🚀 Step 3: Execute Copy Process
**Instructions:**
Run this cell to start the actual copying. It uses the paths you confirmed in Step 2.
* You will see a real-time log of every file being processed.
* If the target folder name already exists, it will stop to prevent overwriting.

In [ ]:
import shutil
import os
from tqdm.notebook import tqdm

# --- Configuration ---
BUFFER_SIZE = 1024 * 1024  # 1 MB chunks (Optimized for Drive)

def get_total_directory_size(start_path):
    """Calculates the total size of the source directory in bytes."""
    total_size = 0
    print("🔍 Calculating total size... (This might take a moment)")
    for dirpath, dirnames, filenames in os.walk(start_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            # Skip if it is symbolic link
            if not os.path.islink(fp):
                total_size += os.path.getsize(fp)
    return total_size

# --- Custom Copy Function with Progress Bar ---
def copy_file_chunked(src, dst, pbar):
    """
    Copies a file from src to dst in chunks, updating the global pbar.
    """
    try:
        with open(src, 'rb') as fsrc:
            with open(dst, 'wb') as fdst:
                while True:
                    buf = fsrc.read(BUFFER_SIZE)
                    if not buf:
                        break
                    fdst.write(buf)
                    pbar.update(len(buf))

        # Attempt to preserve metadata (timestamp), but ignore errors if Drive refuses
        try:
            shutil.copystat(src, dst)
        except:
            pass

    except Exception as e:
        print(f"\n❌ Error copying file {src}: {e}")

# --- Main Logic ---
if 'FINAL_SOURCE_PATH' not in globals() or 'FINAL_TARGET_PATH' not in globals():
    print("⛔ STOP: Variables not defined. Please run Step 2 first.")
else:
    if os.path.exists(FINAL_TARGET_PATH):
        print(f"❌ ERROR: The folder '{os.path.basename(FINAL_TARGET_PATH)}' already exists!")
        print("👉 Please go back to Step 2 and choose a new name.")
    else:
        try:
            # 1. Calculate Total Size for the Progress Bar
            total_bytes = get_total_directory_size(FINAL_SOURCE_PATH)

            print(f"📦 Total Size to Copy: {total_bytes / (1024**3):.2f} GB")
            print(f"📂 Source: {FINAL_SOURCE_PATH}")
            print(f"📂 Target: {FINAL_TARGET_PATH}")
            print("=" * 60)

            # 2. Initialize Progress Bar
            with tqdm(total=total_bytes, unit='B', unit_scale=True, desc="Copying") as pbar:

                # 3. Define a helper to pass to copytree
                # shutil.copytree expects a function that takes (src, dst)
                # We wrap our chunked function to include the pbar
                def _wrapper_copy(src, dst):
                    copy_file_chunked(src, dst, pbar=pbar)

                # 4. Start the Recursive Copy
                shutil.copytree(FINAL_SOURCE_PATH, FINAL_TARGET_PATH, copy_function=_wrapper_copy)

            print("\n" + "=" * 60)
            print("✅ SUCCESS! File transfer(s) completed.")
            print(f"🎉 Files are located at: {FINAL_TARGET_PATH}")
            print("=" * 60)

        except Exception as e:
            print(f"\n❌ UNEXPECTED ERROR: {e}")